In [29]:
import numpy as np
import pandas as pd
import glob
from datetime import timedelta

In [102]:
def is_nan(x):
    '''
    Checks if x is nan or not.
    Taken from https://stackoverflow.com/questions/944700/how-can-i-check-for-nan-values.

    :param x: any type of item to check for if it is nan type

    Returns True if it is nan type, false if not.
    '''

    return (x is np.nan or x != x)

In [62]:
my_list = [1,2,3]
my_list[2] = 'a'
my_list

[1, 2, 'a']

In [30]:
filename_patient = './patientData/patientAS14.01.csv'

In [58]:
df_patient = pd.read_csv(filename_patient)
# Edit dataframe
#df_patient['month'] = df_patient.apply(lambda row: row.time[5:7], axis=1)
#df_patient['day'] = df_patient.apply(lambda row: row.time[8:10], axis=1)
df_patient['call'].fillna(0, inplace=True)
df_patient['sms'].fillna(0, inplace=True)
df_patient['time'] = pd.to_datetime(df_patient['time'])
df_patient.set_index(['time'], inplace=True)
df_patient

,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
time,,,,,,,,,,,,,,,,,,,
2014-02-26,NaN,-0.250000,0.750000,6.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0
2014-02-27,NaN,0.000000,0.333333,6.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0
2014-03-20,0.081548,NaN,NaN,NaN,248.979,2168.229,350.856,NaN,NaN,NaN,11.345,807.731,NaN,45.173,21.074,NaN,1.0,2275.944000,0.0
2014-03-21,0.134050,0.200000,0.200000,6.200000,3139.218,6280.890,1007.456,49.544,NaN,172.206,239.751,4508.500,915.445,NaN,598.754,NaN,6.0,17978.907000,0.0
2014-03-22,0.236880,0.600000,0.500000,6.400000,731.429,4962.918,93.324,21.076,NaN,NaN,98.143,439.632,37.305,NaN,117.621,NaN,3.0,6142.161000,1.0
2014-03-23,0.142741,0.200000,0.800000,6.800000,1286.246,5237.319,94.346,43.403,NaN,NaN,72.823,900.839,NaN,NaN,30.086,30.386,0.0,6773.832001,0.0
2014-03-24,0.078961,0.800000,0.000000,6.000000,866.956,9270.629,976.971,34.106,NaN,3.010,66.558,3223.626,419.805,NaN,178.732,NaN,10.0,15047.351001,0.0
2014-03-25,0.098374,0.500000,0.500000,6.750000,1032.768,10276.751,68.206,43.054,NaN,NaN,178.819,1919.471,NaN,235.223,222.893,NaN,0.0,21475.354999,1.0
2014-03-26,0.101308,-0.200000,0.600000,6.600000,1167.497,8988.753,910.479,52.331,NaN,NaN,97.498,4592.059,NaN,NaN,33.365,NaN,0.0,16423.801000,0.0


In [45]:
instances = [[]]*5
instances

[[], [], [], [], []]

In [108]:
number_of_days = 6
all_instances = []
all_labels = []
for date_0, row_0 in df_patient.iterrows():
    valid_data_count = 0
    instance = [[]]*(number_of_days-1)
    for date, row in df_patient.iterrows():
        for delta_days in range(number_of_days):
            if date_0 - timedelta(days=delta_days) == date:
                instance[delta_days-1] = row['mood']
                if not is_nan(row['mood']):
                    valid_data_count += 1
    if valid_data_count == 6:
        all_labels.append(row_0['mood'])
        all_instances.append(instance)
    
assert len(all_labels)==len(all_instances), 'Amount of instances and amount of labels differ.'
assert len(all_instances) > 0, 'No instances were created.'

all_instances
#all_labels

[[6.75, 6.0, 6.8, 6.4, 6.6],
 [6.6, 6.75, 6.0, 6.8, 7.0],
 [7.0, 6.6, 6.75, 6.0, 6.4],
 [6.4, 7.0, 6.6, 6.75, 8.0],
 [8.0, 6.4, 7.0, 6.6, 7.5],
 [7.5, 8.0, 6.4, 7.0, 7.4],
 [7.4, 7.5, 8.0, 6.4, 6.0],
 [6.0, 7.4, 7.5, 8.0, 6.5],
 [6.5, 6.0, 7.4, 7.5, 6.4],
 [6.4, 6.5, 6.0, 7.4, 6.2],
 [6.2, 6.4, 6.5, 6.0, 6.8],
 [6.8, 6.2, 6.4, 6.5, 6.5],
 [6.5, 6.8, 6.2, 6.4, 6.5],
 [6.5, 6.5, 6.8, 6.2, 6.6],
 [6.6, 6.5, 6.5, 6.8, 7.4],
 [7.4, 6.6, 6.5, 6.5, 7.4],
 [7.4, 7.4, 6.6, 6.5, 7.0],
 [7.0, 7.4, 7.4, 6.6, 7.25],
 [7.25, 7.0, 7.4, 7.4, 8.0],
 [8.0, 7.25, 7.0, 7.4, 7.2],
 [7.2, 8.0, 7.25, 7.0, 6.4],
 [6.4, 7.2, 8.0, 7.25, 7.2],
 [7.2, 6.4, 7.2, 8.0, 6.6],
 [6.6, 7.2, 6.4, 7.2, 6.8],
 [6.8, 6.6, 7.2, 6.4, 7.8],
 [7.8, 6.8, 6.6, 7.2, 7.25],
 [7.25, 7.8, 6.8, 6.6, 7.6],
 [7.6, 7.25, 7.8, 6.8, 7.4],
 [7.4, 7.6, 7.25, 7.8, 7.6],
 [7.6, 7.4, 7.6, 7.25, 7.6],
 [7.6, 7.6, 7.4, 7.6, 7.2],
 [7.2, 7.6, 7.6, 7.4, 7.6],
 [7.6, 7.2, 7.6, 7.6, 7.5],
 [7.5, 7.6, 7.2, 7.6, 7.6],
 [7.6, 7.5, 7.6, 7.2, 7.0],
 [7.0,

## Training models

In [122]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [109]:
X_train, X_test, y_train, y_test = train_test_split(all_instances, all_labels, test_size=0.33, random_state=42)

In [121]:
# Make instance of model
clf = svm.SVR().fit(X_train, y_train)
y_pred = clf.predict(X_test)

C:\Users\quiri\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [120]:
y_test

[7.2, 7.0, 7.4, 7.6, 7.5, 6.5, 7.6, 7.4, 8.0, 6.0, 7.25, 6.2, 6.6, 7.6]

## Evaluation

In [123]:
mean_squared_error(y_test, y_pred)

0.02861306961721372